# Llama 3

```{note}
This paper presents a
new set of foundation models, called [Llama 3](https://arxiv.org/abs/2407.21783). It is a herd of language models that natively support
multilinguality, coding, reasoning, and tool usage. Our largest model is a dense Transformer with
405B parameters and a context window of up to 128K tokens. We find that Llama 3 delivers comparable quality to leading language
models such as GPT-4 on a plethora of tasks.
```

```{figure} ../images/llama3-1.png
```

## Post-Training

We produce the aligned Llama 3 models by applying several rounds of post-training. Each
round of post-training involves supervised finetuning (SFT) followed by Direct Preference Optimization (DPO{cite}`rafailov2024directpreferenceoptimizationlanguage`) on examples collected either via human annotations or generated synthetically.

### Modeling

```{figure} ../images/llama3-2.png
```

#### Chat Dialog Format

To tune LLMs for human-AI interaction, we need to define a chat dialog protocol for the model to understand
human instructions and perform conversational tasks.

```{figure} ../images/llama3-tokenizer.svg
```

#### Reward Modeling

We train a reward model (RM) covering different capabilities on top of the pre-trained checkpoint. We use `all` of our preference data for reward
modeling after filtering out samples with similar responses.

#### Supervised Finetuning

The reward model is then used to perform rejection sampling on our human annotation prompts, Together with this rejection-sampled data and other data sources
(including synthetic data), we finetune the pre-trained language model using a standard cross entropy loss
on the target tokens (while masking loss on prompt tokens).

#### Direct Preference Optimization

We further train our SFT models with DPO for human
preference alignment. For training, we primarily use the `most recent` batches of preference data collected using
the best performing models from the previous alignment rounds.
the following algorithmic modifications to DPO:

* Masking out formatting tokens in DPO loss
* Regularization with NLL loss (DPOP)

#### Model Averaging

Finally, we average models obtained from experiments using various versions of data or hyperparameters at
each RM, SFT, or DPO stage.

#### Iterative Rounds

Following Llama 2, we apply the above methods in six rounds. In each cycle, we collect new preference
annotations and SFT data, sampling synthetic data from the latest models.

### Post-training Data

#### Preference Data

We deploy multiple models for annotation after
each round and sample two responses from `two different models` for each user prompt. These models can
be trained with different data mixes and alignment recipes, allowing for different capability strength (e.g.,
code expertise) and increased data diversity. We ask `annotators` to rate the strength of their preference by
categorizing it into one of four levels, based on how much more they prefer the chosen response over the
rejected one: significantly better, better, slightly better, or marginally better. We also incorporate an editing
step after preference ranking to encourage annotators to further improve the preferred response. Annotators
edit the chosen response directly or prompt the model with feedback to refine its own response. Consequently,
a portion of our preference data has three responses ranked $(edited > chosen > rejected)$.

```{caution}
How to collect multi-turn preference data from two different models? One possible solution:<br>
1. $query_1 \to (chosen_1, rejected_1)$
2. $answer_1 := chosen_1$
3. $(query_1, answer_1, query_2) \to (chosen_2, rejected_2)$
```

In each round of post-training, we use all the preference data that is available at the time for reward modeling,
while only using the latest batches from various capabilities for DPO training. For both reward modeling and
DPO, we use samples that are labeled as the chosen response being significantly better or better than the
rejected counterpart for training and `discard samples with similar responses`.

#### SFT Data

Our finetuning data is largely comprised of the following sources:

* Prompts from our human annotation collection with rejection-sampled responses
* Synthetic data targeting specific capabilities (See [](llama3_capabilities) for more details)
* Small amounts of human-curated data

**Rejection sampling.** During rejection sampling (RS), for each prompt collected during human annotation we sample $K$ (typically between 10 and 30) outputs from the latest chat model policy (usually
the best performing checkpoint from the previous post-training iteration, or the best performing checkpoint
for a particular capability) and use our reward model to select the best candidate. 

```{tip}
In later rounds of post-training, we introduce system prompts to steer RS responses to conform with
desirable tone, style, or formatting, which might be different for different capabilities.
```

**Overall data composition.** In [](llama3_data_process) we describe techniques for categorizing topic, complexity, and quality of our data
samples. In each round of post-training, we adjust our overall data mix carefully across these axes to tune
performance across a wide range of benchmarks.

(llama3_data_process)=
#### Data Processing and Quality Control

Given that most of our training data is model-generated, it requires careful cleaning and quality control.

* **Topic classification:** We first finetune Llama 3 8B into a topic classifier, and perform inference over
all data to classify it into both coarsely-grained buckets (“mathematical reasoning”) and fine-grained buckets (“geometry and trigonometry”).

* **Quality scoring:** We use both reward model and Llama-based signals to obtain a quality score for each
sample. For an RM-based score, we consider data that is in the top quartile of RM scores as high quality.
For a Llama-based score, we prompt Llama 3 checkpoint to rate each sample on a three-point scale for
general English data and a two-point scale
for coding data (Bug Identification and User Intention), and consider samples that obtain the maximum
score as high quality. The RM and Llama-based scores have high disagreement rates, and we find that
combining these signals yield the best recall on our internal test set. Ultimately, we select examples
that are marked as high quality by the RM `or` the Llama-based filter.

* **Difficulty scoring:** Because we are also interested in prioritizing examples that are more complex for
the model, we score data using two measures of difficulty: Instag{cite}`lu2023instaginstructiontagginganalyzing` and Llama-based
scoring.

* **Semantic deduplication:** Finally, we perform semantic deduplication. We first cluster complete dialogs using RoBERTa and within each cluster
sort them by quality score $\times$ difficulty score. We then do greedy selection by iterating through all sorted
examples, and only keeping the ones that have maximum cosine similarity less than a threshold to the
examples seen so far in the cluster.

(llama3_capabilities)=
### Capabilities

#### Code

Here, we present our work on improving coding capabilities via training a code expert, generating synthetic data for SFT, improving formatting
with system prompt steering, and creating quality filters to remove bad samples from our training data.

**Expert training.** We train a code expert which we use to collect high quality human annotations for code
throughout subsequent rounds of post-training. This is accomplished by branching the main pre-training run
and continuing pre-training on a 1T token mix of mostly (>85%) code data. For the last several
thousand steps of training we perform long-context finetuning (LCFT) to extend the expert’s context length
to 16K tokens on a high quality mix of repo-level code data. Finally, we follow the similar post-training
modeling recipes described earlier to align this model, except with SFT and DPO data mixes primarily
targeting code.

```{tip}
The code expert model is used for both preference data annotation (recall that responses of preference data are sampled from two different models) and rejection sampling.
```

**Synthetic data generation.** We use Llama 3 and the code expert to generate a large quantity of synthetic SFT dialogs. We describe three high-level approaches for generating synthetic code data used during SFT.

1. **Synthetic data generation: execution feedback.** The 8B and 70B models show significant performance
improvements when trained on data generated by a larger, more competent model. However, our initial
experiments revealed that training Llama 3 405B on its own generated data is not helpful (and can
even degrade performance). To address this limitation, we introduced execution feedback as a source of
truth, in particular, we generate large
dataset of approximately one million synthetic coding dialogues using the following process:

    * **Problem description generation:** Magicoder{cite}`wei2024magicoderempoweringcodegeneration`.
    
    * **Solution generation:** Then, we prompt Llama 3 to solve each problem in a given programming language.
      ```{tip}
      We observe that adding general rules of good programming to the prompt improves the generated solution quality. Also, we find it is helpful to require the model to explain its thought process in comments.
      ```

    * **Correctness analysis:** We extract the source code from the generated solution and applied a combination of static and dynamic analysis techniques to test its correctness, including:
  
        * **Static analysis:** We run all generated code through a parser and a linter to ensure syntactic correctness.
        
        * **Unit test generation and execution:** For each problem and solution, we prompt the model to generate unit tests, executed in a containerized environment together with the solution, catching run-time execution errors and some semantic errors.

     * **Error feedback and iterative self-correction:** When a solution fails at any step, we prompt the model to revise it. The prompt included the original problem description, the faulty solution, and feedback from the parser/linter/tester. After a unit test execution failure, the model could either fix the code to pass the existing tests or modify its unit tests to accommodate the generated code. Only dialogs that pass all checks are included in the final dataset, used for supervised finetuning (SFT).
  
     * **Fine-tuning and iterative improvement:** The finetuning process is conducted over multiple rounds, with each round building on the previous one. After each round, the model is improved, generating higher-quality synthetic data for the next round.
  
2. **Synthetic data generation: programming language translation.** We observe a performance gap between
major programming languages (e.g., Python/C++) and less common ones (e.g., Typescript/PHP). This
is not surprising as we have less training data for less common programming languages. To mitigate
this, we supplement our existing data by translating data from common programming languages to
less common languages. This is achieved
by prompting Llama 3 and ensuring quality via syntax parsing, compilation, and execution.

3. **Synthetic data generation: backtranslation.** To improve certain coding capabilities (e.g., documentation,
explanations and debugging) where execution feedback is less informative for determining quality, we employ an
alternative multi-step approach. Beginning with code
snippets from a variety of languages in our pre-training data:

    * **Generate:** We prompt Llama 3 to generate data that represents our target capability (e.g., we ask the model to explain a piece of code).
    
    * **Backtranslate:** We then prompt the model to “backtranslate” the synthetically generated data to the original code (e.g., we ask the model to generate code only from its explanation).
  
    * **Filter:** Using the original code as a reference, we prompt the Llama 3 to determine the quality of the output (e.g., we ask the model how faithful the backtranslated code is to the original). We then use the generated examples that have the highest self-verification scores in SFT.
  
**System prompt steering during rejection sampling.** During the rejection sampling process, we used code specific
system prompts to improve code readability, documentation, thoroughness, and specificity.

```{figure} ../images/llama3-3.png
```

**Filtering training data with execution and model-as-judge signals.** We occasionally
encounter quality issues in our rejection-sampled data. Detecting these
issues in our rejection-sampled data is not as straightforward as it is for our synthetic code data, as the
rejection-sampled responses typically contain a mix of natural language and code for which the code may not always be expected to be executable. To address this, we utilize the “model-as-judge” approach,
where earlier versions of Llama 3 assess and assign a binary (0/1) score based on two criteria: code correctness
and code style. We retain only those samples that achieve a perfect score of 2. Initially, this stringent filtering
led to a regression in downstream benchmark performance, primarily because it disproportionately `removed
examples with challenging prompts`. To counteract this, we strategically revise the responses of some coding
data categorized as most challenging until they met the Llama-based “model-as-judge” criteria. By refining
these challenging problems, the coding data achieves a balance between quality and difficulty, resulting in
optimal downstream performance.

```{caution}
How to revise the responses automatically?
```

## Results

### Pre-trained Language Model

```{figure} ../images/llama3-4.png
---
height: 400px
---
```
```{figure} ../images/llama3-5.png
```
```{figure} ../images/llama3-6.png
```

### Post-trained Language Model

```{figure} ../images/llama3-7.png
```

## Takeaway

```{note}
Preference data:
* Response from two different models with different data mixes and alignment recipes
* Human annotate significantly better, better, slightly better, or marginally better. Only use significantly better and better.
* Discard samples with similar responses

SFT data:
* Rejection sampling using RM, system prompt steering during rejection sampling.
* Synthetic data: 
    * Execution feedback (Magicoder)
    * Programming language translation
    * Backtranslation (code explain, debug)

Data process and quality control:
* Quality scoring
    * LLM as judge (for challenging prompts, revise responses until met criteria)
    * Or top quartile of RM
* Difficulty scoring
* Semantic deduplication
```